In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
print(tf.version.VERSION)

1.14.0


# Set-Up Dataset

In [3]:
labels = ['A-BLUE', 'A-RED', 'B-BLUE', 'B-RED']
img_cols = 160
img_rows = 120

def read_img(path, img, class_num, data):
    try:
        img_arr = cv2.imread(os.path.join(path, img)) # read image with opencv
        if img_arr.shape[0] == img_rows and img_arr.shape[1] == img_cols:
            data.append([img_arr, class_num]) # add to final data list
        else:
            print('File at', os.path.join(path,img), 'does not have proper size')
            print('Expected ({}x{}}) but got ({}x{})'.format(img_rows,img_cols,img_arr.rows,img_arr.cols))
    except Exception as e:
        print(e)

def read_data(data_dir, train_set_size_per_cat, val_set_size_per_cat):
    train_data = []
    val_data = []
    
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        
        images = os.listdir(path) # list of images in directory
        training_images = random.sample(images, train_set_size_per_cat) # select 40 random images for training
        images = [image for image in images if image not in training_images] # remove selected images
        validation_images = random.sample(images, val_set_size_per_cat) # select 10 random validation images
        
        # read the training images
        for img in training_images:
            read_img(path, img, class_num, train_data)
                
        # read the validation images
        for img in validation_images:
            read_img(path, img, class_num, val_data)
    
    return np.array(train_data), np.array(val_data)

In [4]:
train, val = read_data('input', 40, 10)

x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

# normalize
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

y_train = np.array(y_train)
y_val = np.array(y_val)

y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

/home/edurso/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
datagen = ImageDataGenerator(
        featurewise_center             =False,  # set input mean to 0 over the dataset
        samplewise_center              =False,  # set each sample mean to 0
        featurewise_std_normalization  =False,  # divide inputs by std of the dataset
        samplewise_std_normalization   =False,  # divide each input by its std
        zca_whitening                  =False,  # apply ZCA whitening
        rotation_range                 =3,      # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range                     =0.1,    # Randomly zoom image 
        width_shift_range              =0.1,    # randomly shift images horizontally (fraction of total width)
        height_shift_range             =0.1,    # randomly shift images vertically (fraction of total height)
        horizontal_flip                =False,  # randomly flip images
        vertical_flip                  =False)  # randomly flip images

datagen.fit(x_train)

# Create and Train Model

In [6]:
def create_model(rows, cols):
    model = tf.keras.models.Sequential([
        keras.layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(rows,cols,3)),
        keras.layers.MaxPool2D(),
        keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(),
        keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(),
        keras.layers.Dropout(0.2),
        keras.layers.Flatten(),
        keras.layers.Dense(112, activation="relu"),
        keras.layers.Dense(56, activation="relu"),
        keras.layers.Dense(56, activation="relu"),
        keras.layers.Dense(4, activation="softmax"),
    ])
    opt = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = create_model(img_rows, img_cols)
history = model.fit(x_train, y_train, epochs=25, validation_data=(x_val, y_val), shuffle=True, verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 160 samples, validate on 40 samples
Epoch 1/25
160/160 - 9s - loss: 1.3832 - acc: 0.2375 - val_loss: 1.3431 - val_acc: 0.5500
Epoch 2/25
160/160 - 8s - loss: 1.2893 - acc: 0.6125 - val_loss: 1.2005 - val_acc: 0.3750
Epoch 3/25
160/160 - 7s - loss: 1.0969 - acc: 0.5437 - val_loss: 0.8983 - val_acc: 1.0000
Epoch 4/25
160/160 - 8s - loss: 0.6328 - acc: 0.9375 - val_loss: 0.2624 - val_acc: 1.0000
Epoch 5/25
160/160 - 7s - loss: 0.2084 - acc: 0.9312 - val_loss: 0.0928 - val_acc: 1.0000
Epoch 6/25
160/160 - 7s - loss: 0.0423 - acc: 0.9937 - val_loss: 0.0086 - val_acc: 1.0000
Epoch 7/25
160/160 - 7s - loss: 0.0073 - acc: 1.0000 - val_loss: 0.0080 - val_acc: 1.0000
Epoch 8/25
160/160 - 6s - loss: 0.0021 - acc: 1.0000 - val_loss: 4.3716e-04 - val_acc: 1.0000
Epoch 9/25
160/160 - 7s - loss: 4.4306e-04 - acc: 1.0000 - val_loss: 3.4861e-04 - val_acc: 1.0000
Epoch 10/25
160

In [8]:
loss, acc = model.evaluate(x_val, y_val, verbose=2)

40/40 - 0s - loss: 6.6369e-06 - acc: 1.0000


# Inference Function

In [9]:
def inference(model, img, expected):
    res = model.predict(img)
    real = expected
    
    int_res_one_hot = [int(round(r)) for r in res[0]]
    int_real_one_hot = [int(round(r)) for r in real]
    #print('Encoded Result: {} | Encoded Expected: {}'.format(int_res_one_hot, int_real_one_hot))
    
    res_idx = -1
    real_idx = -1
    res_ = 'NONE'
    real_ = 'NONE'
    
    try:
        
        res_idx = int_res_one_hot.index(1)
        real_idx = int_real_one_hot.index(1)
        #print('Result Index: {} | Expected Index: {}'.format(res_idx, real_idx))
        
        res_ = labels[res_idx]
        real_ = labels[real_idx]
        #print('Result Index: {} | Expected Index: {}'.format(res_, real_))
        
    except:
        print('NONE')    
    
    prediction = res_
    expected = real_
    correct = (prediction == expected)
    
    return prediction, expected, correct

# Run Inference On A Few Random Images

In [10]:
data, _ = read_data('input', 1, 0) # read one image from each category

x = []
y = []

for feature, label in data:
    x.append(feature)
    y.append(label)
    
x = np.array(x) / 255
y = keras.utils.to_categorical(np.array(y))
    
for img, real in zip(x, y):
    img.resize(1, img_rows, img_cols, 3) # TODO why is this necessary?
    prediction, expected, correct = inference(model, img, real)
    print('Result Index: {} | Expected Index: {}'.format(prediction, expected))

Result Index: A-BLUE | Expected Index: A-BLUE
Result Index: A-RED | Expected Index: A-RED
Result Index: B-BLUE | Expected Index: B-BLUE
Result Index: B-RED | Expected Index: B-RED


/home/edurso/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# Find Misclassified Samples
If no images are printed, there are no misclassified samples and that is good. 

In [11]:
data, _ = read_data('input', 500, 0) # read 50 images from each category

x_ = []
y_ = []

for feature, label in data:
    x_.append(feature)
    y_.append(label)
    
x_ = np.array(x_) / 255
y_ = keras.utils.to_categorical(np.array(y_))
    
for img, real in zip(x_, y_):
    img.resize(1, img_rows, img_cols, 3) # TODO why is this necessary?
    prediction, expected, correct = inference(model, img, real)
    if not correct:
        message = 'Model predicted "{}" but label is "{}"'.format(prediction, expected)
        print(message)
        plt.figure(figsize=(5,5))
        tmp = img.copy()[0]
        plt.imshow(tmp)
        plt.title(message)

/home/edurso/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# Save Model

In [12]:
model.save('./final-classifier.h5')